In [1]:
#Make necessary imports
import requests
from bs4 import BeautifulSoup
import httpx
import json
import matplotlib.pyplot as plt
from langchain.text_splitter import RecursiveCharacterTextSplitter
from supabase import create_client
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
import os
import openai
import json
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity


Could not import azure.core python package.


In [2]:
# URL of the page you want to scrape
url = 'https://www.geeksforgeeks.org/machine-learning/'


# Send GET request to the webpage
response = requests.get(url)

In [ ]:

# Initialize the Supabase client
supabase_url = "https://mufqacshyjgmzivznuwo.supabase.co"
supabase_key = "Please add your key here"
client = create_client(supabase_url, supabase_key)

# change the table name as required
table_name = "combined_test" 

headers = {
    "apikey": supabase_key,
    "Content-Type": "application/json"
}

endpoint = f"{supabase_url}/rest/v1/combined_test"

# Please add your open ai key
openai.api_key = "Please add your key here"

## Convert data to embeddings

In [4]:
# This method preprocesses a given sentence by tokenizing it, converting it to lowercase,
# and removing any tokens that are not alphabetic or are considered stopwords.
def preprocess_sentence(sentence):
    stop_words = set(stopwords.words('english'))
    # Tokenize the sentence
    tokens = word_tokenize(sentence.lower())
    # Remove non-alphabetic characters and stopwords
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Download required NLTK resources, if not already downloaded
def download_nltk_resources():
    nltk.download('punkt')      # For tokenization
    nltk.download('stopwords')  # For removing stop words

# Ensure NLTK resources are downloaded before proceeding
download_nltk_resources()

# This generator function opens a PDF and yields preprocessed text from each page as a TaggedDocument.
# The TaggedDocument is suitable for training Doc2Vec models in Gensim.
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for i, page in enumerate(reader.pages):
            # Extract text from page and preprocess
            text = page.extract_text()
            sentences = nltk.sent_tokenize(text)
            # Preprocess each sentence and combine into one list of words for the page
            words = [preprocess_sentence(sentence) for sentence in sentences]
            words = [word for sublist in words for word in sublist]  # Flatten the list
            # Yielding a TaggedDocument for each page
            yield TaggedDocument(words=words, tags=[str(i)])
            
pdf_path = 'C:/Users/devas/Downloads/AI-book.pdf'
tagged_data = list(extract_text_by_page(pdf_path))

### Divide the data in chunks of size 1000
def chunk_section(section, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],   # Define separators for splitting text.
        chunk_size=chunk_size,                # Maximum size of each chunk.
        chunk_overlap=chunk_overlap,          # Number of characters to overlap between chunks.
        length_function=len)

    # Use the text splitter to divide the input text into chunks.
    chunks = text_splitter.create_documents(
        texts=[section["text"]], 
        metadatas=[{"source": section["source"]}])
    # Return a list of dictionaries, each containing a chunk of text and its associated metadata.
    return [{"text": chunk.page_content, "source": chunk.metadata["source"]} for chunk in chunks]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\devas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Train and save the model

In [6]:
# Initialize the Doc2Vec model with specific parameters.
# - vector_size: Dimensionality of the feature vectors.
# - window: The maximum distance between the current and predicted word within a sentence.
# - min_count: Ignores all words with total frequency lower than this.
# - workers: Number of worker threads to train the model (faster training with multicore machines).
# - epochs: Number of iterations (passes) over the corpus.
model = Doc2Vec(vector_size=50, window=5, min_count=5, workers=4, epochs=100)

# Build a vocabulary from the tagged data.
model.build_vocab(tagged_data)

# Train the Doc2Vec model on the tagged data.
# - total_examples: Total number of documents.
# - epochs: Number of iterations over the corpus
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Save the model
model.save("jesc102_model.d2v")

In [ ]:
model = Doc2Vec.load("jesc102_model.d2v")

## Load the data to supabase

In [ ]:
sections= []
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the <ul> tag with the class 'leftBarList'
    left_bar_list = soup.find('ul', class_='leftBarList')
    
    # Find all <a> tags within the left bar list
    links = left_bar_list.find_all('a') if left_bar_list else []

    # Loop through all found <a> tags to get their href attribute (URLs)
    for link in links:
        # Ensure that only full URLs are printed
        href = link.get('href')
        full_url = href if href.startswith('http') else f"https://www.geeksforgeeks.org{href}"
        
        page_response = requests.get(full_url)
        if page_response.status_code == 200:
            # Parse the HTML content of the page
            page_soup = BeautifulSoup(page_response.content, 'html.parser')
            
             # Find all <p> tags on the page and get their text content
            paragraphs = page_soup.find_all('p')
            paragraph_texts = [p.get_text(separator='\n', strip=True) for p in paragraphs]
            
            # Combine all paragraph texts into a single string
            full_text = '\n\n'.join(paragraph_texts)

            # create a section with text and source field
            section = {"text": full_text, "source": full_url}

            # combine all sections with their respective sources
            sections.append({"text": full_text, "source": full_url})
            chunks_ds = chunk_section(section,chunk_size=chunk_size, chunk_overlap=chunk_overlap)

            # Load data chunks for each section to Supabase DB
            for chunk in chunks_ds:
                vector = model.infer_vector(preprocess_sentence(chunk["text"])).tolist()
                vector_data = json.dumps(vector)
            
                #Prepare the data to be inserted 
                data_to_insert = {
                    "source": chunk["source"],
                    "text" : chunk["text"],
                    "vector": vector_data 
                }
                
            # Send a POST request to store the vector data
            response = httpx.post(endpoint, headers=headers, json=data_to_insert)
            
        else:
            print(f"Failed to retrieve the page from {full_url}. Status code: {page_response.status_code}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

## LLM config

In [ ]:
# A decorator function to measure and print the execution time of functions.
def time_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"{func.__name__} took {elapsed_time:.6f} seconds to execute.")
        return result
    return wrapper

# Function to continue a conversation using OpenAI's GPT-4.
@time_decorator
def continue_conversation(prompt):
    response = openai.ChatCompletion.create( model="gpt-4", top_p=0,
                                        messages=prompt,
                                        temperature=0 )
    return response['choices'][0]['message']['content'].strip()


## Load knowledge source vectors from dupabase

In [ ]:
# Supabase details
url: str = "https://mufqacshyjgmzivznuwo.supabase.co"
key: str = "Please add your key here"

# Create a client to connect to Supabase
supabase: Client = create_client(url, key)

import ast
import numpy as np
import pandas as pd

ids = []
sources = []
vectors = []

response = supabase.table("vector_final").select("id, source, vector").execute()
if response.data:
    for row in response.data:
        # Extracting each column's value and appending to the respective lists
        ids.append(row['id'])
        sources.append(row['source'])
        vectors.append(row['vector'])  

# # Prepare the vectors for cosine similarity
vectors = [ast.literal_eval(vector) for vector in vectors]
vectors = np.array(vectors)

## Load test files

In [ ]:
# Load ground truth and test data from Excel files
truth = pd.read_excel('llm_testing.xlsx')
test = pd.read_excel('questions.xlsx')

## Evaluation

In [ ]:
# Main process for checking answers against a test dataset
answers = []
for q in test["Question"]:
    top5_data = []
    initial_user_prompt= q
    new_system_prompt ="Answer the question asked by the user" 

    print(q)
    
    history = [
        {"role": "system", "content": new_system_prompt},
        {"role": "user", "content": initial_user_prompt}]

    # Get the answer of question from LLM
    response_by_LLM = continue_conversation(history)
    vector_sentence = model.infer_vector(preprocess_sentence(response_by_LLM))
    sr = []
    
    # Convert the response to vector embeddings
    vector_sentence = np.array(vector_sentence)
    vector_sentence_2d = vector_sentence.reshape(1, -1)

    # calculate the cosine similarity
    similarities = cosine_similarity(vector_sentence_2d, vectors)

    # Find the index of the most similar paragraph
    most_similar_index = np.argmax(similarities)

    top_5_indexes = np.argsort(similarities[0])[-5:][::-1]
    for index in top_5_indexes:
        sr.append(sources[index])

    res = []
    for i in range(len(sr)):
        # fetch data for top 5 contexts from supabase
        data = supabase.table("vector_final").select("text").eq("source", sr[i]).execute()
        top5_data.append(data)

        # Extract the relevant text
        relevant_text_from_source = [row['text'] for row in top5_data[i].data]
    
        initial_user_prompt= q
        
        #Prompt given to GPT4
        new_system_prompt = f"Compare the following response with the provided source text using semantic check and determine its accuracy. " \
                     f"Response: '{response_by_LLM}'. " \
                     f"Source Text: '{relevant_text_from_source}'. " \
                     f"If the response accurately reflects the source text, reply 'accurate'. " \
                     f"If the response does not accurately reflect the source text , reply 'inaccurate'. " \
                     f"If the response's content is not found in the source text, reply 'information not found in source'."\
                     f"If the response is not relevant to question, reply 'response is irrelevant to question'. "


    
        history = [
            {"role": "system", "content": new_system_prompt},
            {"role": "user", "content": initial_user_prompt}]

        response = continue_conversation(history)
        res.append(response)
        print(res)
    
    # If accurate answer is found in any of top 5 contexts, tag the answer as accurate
    if 'Accurate' in res:
        answers.append("Final check: Accurate")
    else:
        answers.append("Final check: "+ response)


## Calculate Accuracy

In [ ]:
## Claculate the accuracy
# Compare answers from LLM with expected answers and calculate the accuracy
acc = 1
for i in range(len(answers)):
    print(truth['Answer'][i])
    print(answers[i])
    if (truth['Answer'][i] == answers[i]):
        acc += 1
print("Accuracy : ", acc/ len(answers))